by **Ariel Rodrigo Colque Garcia**


In [ ]:
!nvidia-smi

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/107.7 GB disk)


In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstal

# Download datase from Roboflow

In [ ]:
  from roboflow import Roboflow
  rf = Roboflow(api_key="oBkRYmxIVRFQKVIvhZVa")
  project = rf.workspace("segmentacion-iwaek").project("medicinal_plants")
  dataset = project.version(15).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.225, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to medicinal_plants-15 in yolov8:: 100%|██████████| 47678/47678 [00:17<00:00, 2655.83it/s]


# Implements Albumentation

In [ ]:
!pip install albumentations

In [ ]:
import cv2
import albumentations as A
import os
import shutil

# Data Augmentation for training data

In [ ]:
import os
image_root = '/content/medicinal_plants-15/train/images'
labels_root = '/content/medicinal_plants-15/train/labels'

a_images = '/content/medicinal_plants-15/train/images'
a_labels = '/content/medicinal_plants-15/train/labels'

In [ ]:
# Obtener la lista de nombres de archivos en la carpeta de imágenes
image_files = os.listdir(image_root)
# Filtrar solo los archivos de imágenes
image_files = [file for file in image_files if file.lower().endswith(('.jpg', '.jpeg'))]

In [ ]:
image_files[:10] ,len(image_files)

(['eucalipto184_jpg.rf.b8b2dd30d5e0217570f824569ca5e20f.jpg',
  'IMG_20230925_151147_947_jpg.rf.ab1e642360c130a0884577201ae9693f.jpg',
  'IMG_20230925_170046_388_jpg.rf.a2be2d70e461e98702818db6163d803d.jpg',
  'malva16_jpg.rf.b43f2ead856c043b3849a0cdcf0548d6.jpg',
  'manzanilla482_jpg.rf.de610c758e77ea905aa0f679ce9a05a6.jpg',
  'IMG_20230921_150517_876_jpg.rf.c70bbaaa1cdbd84e9c4f7e5a925da0e1.jpg',
  'IMG_20231202_125617_137_jpg.rf.69e36a770bb29e281f4279344f295bfd.jpg',
  'IMG_20230921_080859_902_jpg.rf.20cd29dca8a6e186f0a27473d29eee9a.jpg',
  'eucalipto856_jpg.rf.2291a80b1d3f98c9b29183aa1c42cf1b.jpg',
  'eucalipto259_jpg.rf.47d8dbb3d59f07c1f1b222b36a93e6f4.jpg'],
 21304)

In [ ]:
import os
# reemplaza 'ruta_de_tu_carpeta' con la ruta de tu carpeta
num_archivos = len(os.listdir('/content/medicinal_plants-15/train/images'))
print('Número de archivos en la carpeta:', num_archivos)

Número de archivos en la carpeta: 21304


In [ ]:
# Loop over each file in the image directory
for i, file in enumerate(image_files):
    # Read the original image
    image = cv2.imread(f"{image_root}/{file}")

    # Define the transformation for both image and labels
    transform = A.Compose([
        A.RandomBrightnessContrast(p=1),
        A.RandomGamma(p=1),
        A.CLAHE(p=1),
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-shaper.jpg', t_image)

    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-shaper.txt')

    # Define the transformation for both image and labels
    transform = A.Compose([
        A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.2,p=0.5),
        A.MotionBlur(blur_limit=33, p=0.1),
        A.GaussNoise(var_limit=(0, 255), p=0.1)
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-Channel.jpg', t_image)
    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-Channel.txt')

        # Define the transformation for both image and labels
    transform = A.Compose([
        A.CLAHE(p=1),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1),
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-Multi.jpg', t_image)

    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-Multi.txt')

In [ ]:
import os

# reemplaza 'ruta_de_tu_carpeta' con la ruta de tu carpeta
num_archivos = len(os.listdir('/content/medicinal_plants-15/train/images'))
print('Número de Imagenes en la carpeta:', num_archivos)

Número de Imagenes en la carpeta: 85216


In [ ]:
# reemplaza 'ruta_de_tu_carpeta' con la ruta de tu carpeta
num_archivos = len(os.listdir('/content/medicinal_plants-15/train/labels'))
print('Número de Labels en la carpeta:', num_archivos)

Número de Labels en la carpeta: 85216


# Data Augmentation for validate data

In [ ]:
# reemplaza 'ruta_de_tu_carpeta' con la ruta de tu carpeta
num_archivos = len(os.listdir('/content/medicinal_plants-15/valid/images'))

print('Número de Imagenes en la carpeta:', num_archivos)

Número de Imagenes en la carpeta: 1724


In [ ]:
image_root = '/content/medicinal_plants-15/valid/images'
labels_root = '/content/medicinal_plants-15/valid/labels'

a_images = '/content/medicinal_plants-15/valid/images'
a_labels = '/content/medicinal_plants-15/valid/labels'

In [ ]:
# Obtener la lista de nombres de archivos en la carpeta de imágenes
image_files = os.listdir(image_root)
# Filtrar solo los archivos de imágenes
image_files = [file for file in image_files if file.lower().endswith(('.jpg', '.jpeg'))]

In [ ]:
# Loop over each file in the image directory
for i, file in enumerate(image_files):
    # Read the original image
    image = cv2.imread(f"{image_root}/{file}")

    # Define the transformation for both image and labels
    transform = A.Compose([
        A.RandomBrightnessContrast(p=1),
        A.RandomGamma(p=1),
        A.CLAHE(p=1),
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-shaper.jpg', t_image)

    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-shaper.txt')

    # Define the transformation for both image and labels
    transform = A.Compose([
        A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.2,p=0.5),
        A.MotionBlur(blur_limit=33, p=0.1),
        A.GaussNoise(var_limit=(0, 255), p=0.1)
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-Channel.jpg', t_image)
    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-Channel.txt')

        # Define the transformation for both image and labels
    transform = A.Compose([
        A.CLAHE(p=1),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1),
    ])

    # Apply the transformation to the image and labels
    augmented = transform(image=image)
    t_image = augmented['image']

    # Save the transformed image
    cv2.imwrite(f'{a_images}/{file[:-4]}-Multi.jpg', t_image)

    # Copy the corresponding text file to the new directory
    shutil.copy(f'{labels_root}/{file[:-4]}.txt', f'{a_labels}/{file[:-4]}-Multi.txt')

In [ ]:
# reemplaza 'ruta_de_tu_carpeta' con la ruta de tu carpeta
num_archivos = len(os.listdir('/content/medicinal_plants-15/valid/images'))

print('Número de Imagenes en la carpeta:', num_archivos)

Número de Imagenes en la carpeta: 6896


# Save dataset in kaggle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
#files.upload() # sube el archivo kaggle.json desde la máquina local
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle
kaggle.api.authenticate()

In [ ]:
import json

# crea un archivo JSON con los metadatos del conjunto de datos
metadata = {
  "title": "medicinal_plants-15",
  "id": "rodrigocolque/medicinal-plants-15",  # Modificado el "slug"
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [ ]:
json_object =json.dumps(metadata, indent=4)
with open("/content/medicinal_plants-15/dataset-metadata.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
!kaggle datasets create --public --dir-mode zip --path medicinal_plants-15

Starting upload for file train.zip
100% 11.7G/11.7G [01:49<00:00, 115MB/s]
Upload successful: train.zip (12GB)
Starting upload for file valid.zip
100% 944M/944M [00:09<00:00, 108MB/s]
Upload successful: valid.zip (944MB)
Starting upload for file README.roboflow.txt
100% 1.20k/1.20k [00:00<00:00, 1.44kB/s]
Upload successful: README.roboflow.txt (1KB)
Starting upload for file README.dataset.txt
100% 159/159 [00:00<00:00, 191B/s]  
Upload successful: README.dataset.txt (159B)
Starting upload for file test.zip
100% 53.9M/53.9M [00:01<00:00, 47.6MB/s]
Upload successful: test.zip (54MB)
Starting upload for file data.yaml
100% 393/393 [00:00<00:00, 652B/s]
Upload successful: data.yaml (393B)
Your public Dataset is being created. Please check progress at https://www.kaggle.com/datasets/rodrigocolque/medicinal-plants-15
